# Widgets. ¡Aprende a usarlos!

En este documento se muestra una pequeña guía interactiva del uso de los widgets, dando protagonismo al usuario para ejecutar las celdas y poder modificar los parámetros. En el apartado final se presenta un ejemplo de aplicación con un filtro electrónico.

![](jupyter.png) 

## Introducción

Los **widgets** son objetos interactivos de Python que dan al usuario el control sobre su representación, tales como una barra deslizante (slider) o una caja de texto. Son elementos muy útiles que aumentan la **interactividad** de los notebooks, pues gracias a ellos se puede visualizar información de manera mucho más gráfica. Para su uso, es necesario importar la librería **ipywidgets**. Esta librería contiene multitud de funciones muy útiles.

Los widgets tienen su propio sistema de representación, y se dibujan debajo de la celda ejecutada. También se pueden mostrar explícitamente con `display(…)`



## 1. Primeros pasos con widgets...

**Ejecute** la siguiente celda para ver un *Slider*.

In [ ]:
import ipywidgets as widgets
from IPython.display import display
w = widgets.IntSlider()
display(w)

Si se dibujan varias instancias del mismo widget, estas estarán en sincronía unas con otras, de forma que, si varía una, el resultado se mostrará en todas las instancias de ese objeto.

**Ejecute** la siguiente celda y pruebe a variar alguno de los *Sliders*.

In [ ]:
display(w)
display(w)

Cada widget tiene unas **propiedades** que se pueden editar, como valor, descripción, etc. Estos aspectos se pueden indicar en la construcción del objeto o a posteriori. Para listar todas las propiedades se utiliza el atributo ‘keys’.


**Ejecute** la siguiente celda para ver qué atributos tiene el slider que ha creado anteriormente.

In [ ]:
w.keys

## 2. ¿Qué tipos de widgets existen?

Pero existen además otros tipos de widgets: cajas de texto, botones, selectores de color, casillas de selección, etc.

**Ejecute** cada celda de código para ver el resultado.

In [ ]:
widgets.Text(value='Hola Mundo!', disabled=True)

In [ ]:
widgets.BoundedIntText(
    value=7,               # valor inicial
    min=0,                 # límite menor
    max=10,                # límite mayor
    step=1,                # intervalo entre valores
    description='Test',    # descripción
    disabled=False         # deshabilitar interacción
)

In [ ]:
widgets.Checkbox(
    value=False,
    description='Check me',
    disabled=False,
    indent=True             # aplicar sangría
)

In [ ]:
widgets.Dropdown(
    options=['física', 'matemáticas', 'química'],
    value='física',
    description='Asignatura:',
    disabled=False,
)

In [ ]:
widgets.RadioButtons(
    options=['azul', 'verde', 'rojo', 'amarillo'],
    description='color:',
    disabled=False
)

In [ ]:
widgets.ToggleButtons(
    options=['Madrid', 'Barcelona', 'Valencia'],
    description='Ciudad:',
    button_style='info', # 'success', 'info', 'warning', 'danger' o ''
    tooltips=['Comunidad de Madrid', 'Cataluña', 'Comunidad Valenciana']
)

También existe la posibilidad de vincular varios widgets distintos (*jslink o link*) para mostrar el mismo valor de distintas maneras.

In [ ]:
a = widgets.FloatText()
b = widgets.FloatSlider(
    min=-20,                  
    max=50,                    
    description='¡Muéveme!',   
    step=2)                 
display(a,b)
mylink = widgets.jslink((a, 'value'), (b, 'value'))

O de modificar la presentación de los datos:

In [ ]:
secciones = widgets.Accordion(children=[widgets.ColorPicker(), widgets.Text()])
secciones.set_title(0, 'Elige un color!')
secciones.set_title(1, 'Inserta tu nombre')
secciones

In [ ]:
secciones.selected_index = 0     # Al ejecutar la celda, se controla qué 
                                 # sección estará visible para el usuario.
                                 # Varíe el parámetro entre 0 y 1

## 3. Widgets de salida

Un widget de salida puede almacenar y mostrar todo lo que acumule stdout y stderr: texto, ecuaciones, vídeos, gráficas, etc.

In [ ]:
import ipywidgets as widgets
from IPython.display import display
out = widgets.Output(layout={'border': '2px solid red'})
out.append_stdout('Salida personalizada para el usuario:')
display(out)

Es posible añadir contenido directamente sobre un widget de salida según el usuario quiera:

In [ ]:
with out:
    print('')
    print('Los resultados para el experimento son:')
    for i in range(3):
        print(i)

E incluso el valor de retorno de una función con el *decorador* `capture` 

In [ ]:
@out.capture()
def funcion_con_salida_capturada():
    print('Texto almacenado por out')

funcion_con_salida_capturada()

In [ ]:
out.clear_output()

## 4. Un paso más allá...

La función **interact** crea un componente con el que controlar argumentos de una función, para posteriormente llamar a esa función con los argumentos manipulados interactivamente. Por ejemplo:

In [ ]:
from ipywidgets import interact

def cuadrado(x):
    return x*x

interact(cuadrado, x=10);

La función **interactive** es parecida a la anterior, pero en este caso el valor de retorno es un widget. Esto permite reusar los widgets o los datos. Además este widget resultante tiene atributos: `children` (elementos contenidos), `kwargs` (argumentos), `result` (resultado)...

In [ ]:
from ipywidgets import interactive
from IPython.display import display

def sumatorio(a, b):
    display (a+b)
    return a+b

w=interactive(sumatorio, a=10, b=20)
display(w)

Otro derivado de `interact` es **interactive_output**, que consigue conectar los controles de los sliders, con una función cuyo valor de retorno es capturado en el widget de salida.

In [ ]:
from ipywidgets import interactive_output

a = widgets.IntSlider(description='a')
b = widgets.IntSlider(description='b')
c = widgets.IntSlider(description='c')
def sumatorio(a_i, b_i, c_i):
    print('{}+{}+{}={}'.format(a_i, b_i, c_i, a_i+b_i+c_i))

layout = widgets.interactive_output(sumatorio, {'a_i': a, 'b_i': b, 'c_i': c})

widgets.HBox([widgets.VBox([a, b, c]), layout])

## 5. ¡Manos a la obra!

Veamos ahora un ejemplo de aplicacióna a la **electrónica**. Se muestra una función que calcula la respuesta en frecuencia de un Filtro de Butterworth Paso Bajo, para posteriormente representarlo gráficamente.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.signal as sig
from ipywidgets import interact, IntSlider

In [ ]:
# Respuesta en frecuencia de un filtro de Butterworth Paso Bajo

@interact(N=IntSlider(min=1, max=10, step=1, value=8),
          Wn=IntSlider(min=1, max=1000, step=100, value=100))
def represent(N, Wn):
    b, a = sig.butter(N=N, Wn=Wn, btype='lowpass', analog=True)

    w, h = sig.freqs(b, a)

    plt.figure(1, figsize=(10,4))
    plt.semilogx(w, 20 * np.log10(abs(h)))
    plt.title('Respuesta en frecuencia del Filtro de Butterworth')
    plt.xlabel('Frecuencia [radianes / segundo]')
    plt.ylabel('Amplitud [dB]')
    plt.margins(0, 0.1)
    plt.grid(which='both', axis='both')
    plt.axvline(Wn ,color='green') # frecuencia de corte
    plt.show(plt.figure(1))


    plt.figure(2, figsize=(10,4))
    plt.semilogx(w, np.arctan2(np.imag(h), np.real(h)))
    plt.title('Respuesta en fase del Filtro de Butterworth')
    plt.xlabel('Frecuencia [radianes / segundo]')
    plt.ylabel('Fase [radianes]')
    plt.margins(0, 0.1)
    plt.grid(which='both', axis='both')
    plt.show(plt.figure(2))

**Varíe** los *Sliders* para modificar el valor del orden y la frecuencia de corte del filtro.